In [3]:
import numpy as np
import scipy.io

def _bin(k, N):
    return (k + N) % N

def make_ltf_xcp2(Nfft=64, used_lo=-26, used_hi=26, Ncp=16, seed=7):
    rng = np.random.default_rng(seed)
    X = np.zeros(Nfft, dtype=np.complex64)
    for k in range(used_lo, used_hi + 1):
        if k == 0:
            continue
        X[_bin(k, Nfft)] = 1 - 2 * rng.integers(0, 2)  # ±1 BPSK

    # CP-less LTF
    x_no_cp = np.fft.ifft(np.fft.ifftshift(X)).astype(np.complex64)
    x_no_cp /= np.sqrt(np.mean(np.abs(x_no_cp)**2) + 1e-12)

    # Add CP
    cp = x_no_cp[-Ncp:]
    x_cp1 = np.concatenate([cp, x_no_cp])
    x_cp2 = np.concatenate([x_cp1, x_cp1])
    return x_cp2.astype(np.complex64)

# Step 1: Generate LTF (CP+LTF, CP+LTF)
ltf_sequence = make_ltf_xcp2(Nfft=64, used_lo=-26, used_hi=26, Ncp=16, seed=7)

# Step 2: Save to preamble.mat with variable name 'ltf' (to match your loader)
scipy.io.savemat("preamble.mat", {"ltf": ltf_sequence.reshape(-1, 1)})
print("Saved preamble.mat with variable 'ltf'.")

# Use Cases Below
def generate_ltf():
    mat = scipy.io.loadmat('preamble.mat')
    ltf = mat['ltf'].flatten()  # complex array
    return np.tile(ltf, 1)      # repeat 1 times

# Step 4: Get the full IQ array
iq_samples = generate_ltf()
print(f"IQ samples shape: {iq_samples.shape}")


Saved preamble.mat with variable 'stf'.
IQ samples shape: (320,)
